In [2]:
try:
    import evidently
except: 
    %pip install git+https://github.com/evidentlyai/evidently.git

In [3]:
import pandas as pd 

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset

In [4]:
%pip install ucimlrepo

In [5]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
spambase = fetch_ucirepo(id=94) 
  
# data (as pandas dataframes) 
X = spambase.data.features 
y = spambase.data.targets 
df = pd.concat([X,y], axis=1)
print(df)

      word_freq_make  word_freq_address  word_freq_all  word_freq_3d  \
0               0.00               0.64           0.64           0.0   
1               0.21               0.28           0.50           0.0   
2               0.06               0.00           0.71           0.0   
3               0.00               0.00           0.00           0.0   
4               0.00               0.00           0.00           0.0   
...              ...                ...            ...           ...   
4596            0.31               0.00           0.62           0.0   
4597            0.00               0.00           0.00           0.0   
4598            0.30               0.00           0.30           0.0   
4599            0.96               0.00           0.00           0.0   
4600            0.00               0.00           0.65           0.0   

      word_freq_our  word_freq_over  word_freq_remove  word_freq_internet  \
0              0.32            0.00              0.00     

In [6]:
# Split your data into reference and current datasets
reference_df = df.sample(frac=0.3, random_state=1)
current_df = df.drop(reference_df.index)

In [7]:
from evidently import ColumnMapping

column_mapping = ColumnMapping()

# Define categorical features
column_mapping.categorical_features = []

# Define numerical features
column_mapping.numerical_features = [
"word_freq_make",
"word_freq_address",
"word_freq_all",
"word_freq_3d",
"word_freq_our",
"word_freq_over",
"word_freq_remove",
"word_freq_internet",
"word_freq_order",
"word_freq_mail",
"word_freq_receive",
"word_freq_will",
"word_freq_people",
"word_freq_report",
"word_freq_addresses",
"word_freq_free",
"word_freq_business",
"word_freq_email",
"word_freq_you",
"word_freq_credit",
"word_freq_your",
"word_freq_font",
"word_freq_000",
"word_freq_money",
"word_freq_hp",
"word_freq_hpl",
"word_freq_george",
"word_freq_650",
"word_freq_lab",
"word_freq_labs",
"word_freq_telnet",
"word_freq_857",
"word_freq_data",
"word_freq_415",
"word_freq_85",
"word_freq_technology",
"word_freq_1999",
"word_freq_parts",
"word_freq_pm",
"word_freq_direct",
"word_freq_cs",
"word_freq_meeting",
"word_freq_original",
"word_freq_project",
"word_freq_re",
"word_freq_edu",
"word_freq_table",
"word_freq_conference",
"char_freq_;",
"char_freq_(",
"char_freq_[",
"char_freq_!",
"char_freq_$",
"char_freq_#",
"capital_run_length_average",
"capital_run_length_longest",
"capital_run_length_total"
]

In [8]:
# Create a report with DataDriftPreset
data_drift_report = Report(metrics=[DataDriftPreset(stattest='ks',stattest_threshold=0.03)])
data_drift_report.run(reference_data=reference_df, current_data=current_df, column_mapping=column_mapping)

In [ ]:
# Visualize the report
data_drift_report.save_html("spambase_report.html")

#### Create Custom Test to Integrate Frouros DDM detection for comcept Drift

In [ ]:
from typing import Tuple
from frouros.detectors.concept_drift import DDM, DDMConfig
from evidently.calculations.stattests import StatTest

In [ ]:
def frouros_ddm_drift_test(reference_data: pd.Series, current_data: pd.Series, _feature_type: str, threshold: float) -> Tuple[float, bool]:
    # Detector configuration and instantiation
    config = DDMConfig(
        warning_level=2.0,
        drift_level=3.0,
        min_num_instances=25,  # minimum number of instances before checking for concept drift
    )
    detector = DDM(config=config)
    
    drift_flag = False
    for i, (ref, cur) in enumerate(zip(reference_data, current_data)):
        error = 1 - (cur == ref)
        _ = detector.update(value=error)
        status = detector.status
        if status["drift"] and not drift_flag:
            drift_flag = True
            return i, True  # return the step at which drift was detected and the boolean flag
    return None, False  # return None if no drift was detected


In [ ]:
frouros_ddm_stat_test = StatTest(
    name="frouros_ddm_test",
    display_name="Frouros DDM Test (score)",
    func=frouros_ddm_drift_test,
    allowed_feature_types=["num"],  # replace with the correct feature types
)